In [1]:
## Extracting portions of txt files ref : https://www.computerhope.com/issues/ch001721.htm
## Config
input_txt = r'result_Be10-copy.txt'

In [14]:
## Package used
import os
from pathlib import Path
import re
import pandas as pd

### Functions used
def remove_fermi(energy_list):
    pattern_fermi = re.compile("Fermi")
    for line in energy_list:
        if pattern_fermi.search(line) != None:
            energy_list.remove(line)
    return energy_list

def identify_line(text_file:'str',pattern_start:'str', pattern_end:'str')->tuple:
    pat_start = re.compile(pattern_start)
    pat_end = re.compile(pattern_end)
    line_start_counter = []
    line_end_counter = []
    
    for counter, line in enumerate(text_whole):
        if pat_start.search(line) != None:
            line_start_counter.append(counter)
        else:
            pass
        if pat_end.search(line):
            line_end_counter.append(counter)
        else:
            pass
    output_line_counter = line_start_counter + line_end_counter
#     if len(output_line_counter) >= 3:
#         print('repeated patterns found for: ', pattern_end, ' and ', pattern_start)
#     else:
#         pass
    return output_line_counter

In [3]:
## setting up the path 
main_dir_path = Path(os.getcwd())
assess = str(main_dir_path); assess = (assess.split("\\")); assess.reverse()
while assess[0] != "VQE_pair":
    main_dir_path = main_dir_path.parent
    assess = str(main_dir_path); assess = (assess.split("\\")); assess.reverse()
os.chdir(main_dir_path)
input_txt = os.path.join(main_dir_path, 'Data', input_txt)

### Using a dictionary to store the file names to be used
filenames = {}
#### Input
filenames["input_file"] = input_txt   ## source data file name must be in /Data/source_data.txt
filenames["input_file_split"] = re.split('-|_|\\.', filenames["input_file"])
nucleus_name = filenames["input_file_split"][2]
# #### Output
filenames["output_1B_source_csv"] =  os.path.join(main_dir_path,"Data",nucleus_name+"-1B-source.csv")
filenames["output_2B_source_csv"] =  os.path.join(main_dir_path,"Data",nucleus_name+"-2B-source.csv")
filenames["config_output_py"] = os.path.join(main_dir_path,"Data",nucleus_name+"_num_config.py")

In [4]:
## Reading the txt file and convert to a list object
text_whole = []
with open (filenames["input_file"], 'rt') as myfile:    
    for line in myfile:
        line = line.rstrip('\n')
        text_whole.append(line)

## Extract 2body matrix elements information
extracted_2body = []
pattern_hf = re.compile("HF INDEX AND ISOSPIN")
for line in text_whole:
    if pattern_hf.search(line) != None:      # If a match is found
        extracted_2body.append((line.rstrip('\n')).lstrip('HF INDEX AND ISOSPIN'))
### Store the information extracted into a dataframe; and output them as csv for later viewing/assessment
obs_twobody_df = pd.DataFrame(columns = ["i1","isoi1","i2","isoi2","f1","isof1","f2","isof2","V"])
obs_twobody_df

for counter, extracted_2body_row in enumerate(extracted_2body):
    appending_row = []
    split_string_list = extracted_2body_row.split(" ")
    for split in split_string_list:
        if split =="":
            pass
        else:
            appending_row.append(float(split))
    obs_twobody_df.loc[counter] = appending_row
obs_twobody_df.to_csv(filenames["output_2B_source_csv"])

In [13]:
## Extraction of num_particles data; Create a new class of object called num_orbitals
## Below the number of particles and orbitals all refer to the valence"s
## Valence!! VALENCE!!
pattern_num_particles = re.compile("isospin label Omega pi   energy  occupation partner_n partner_p")
num_particles = [None,None]
num_nucleon_orbitals = [None,None]

## Extract particles number information
pattern_energy_start = "-------------------- Lower bound for Pairing Valence Space -------------------------"
pattern_energy_end   = "-------------------- Upper bound for Pairing Valence Space -------------------------"
energy_block_id = identify_line(text_whole, pattern_energy_start,pattern_energy_end);
energy_block_id.sort()

energy_neut_list = text_whole[energy_block_id[0]+1: energy_block_id[1]]
energy_prot_list = text_whole[energy_block_id[2]+1: energy_block_id[3]]
energy_neut_list = remove_fermi(energy_neut_list)
energy_prot_list = remove_fermi(energy_prot_list)

num_neut_list = []; num_prot_list = []
for line in energy_neut_list:
    num_neut_list.append(int(line[53:55]))
for line in energy_prot_list:
    num_prot_list.append(int(line[53:54]))

num_particles[0] = num_neut = sum(num_neut_list); num_particles[1] = num_prot = sum(num_prot_list)
num_particles = tuple(num_particles)

num_neut_orbitals = energy_block_id[1] - energy_block_id[0]; num_prot_orbitals = energy_block_id[3] - energy_block_id[2]

num_nucleon_orbitals[0] = num_neut_orbitals; num_nucleon_orbitals[1] = num_prot_orbitals
num_nucleon_orbitals = tuple(num_nucleon_orbitals)

num_spin_orbitals = sum(num_nucleon_orbitals)

NameError: name 'blockPrint' is not defined

## onebody matrix informations terms

In [6]:
# Extract onebody information
## Store the onebody terms into panda dataframes and export as csv
obs_onebody_df = pd.DataFrame(columns = ["hf_lvl","iso", "epsilon"])

for counter, line in enumerate(energy_neut_list+energy_prot_list):
    dummy_append = []
    dummy_df_row = []
    split_string_list = line.split("   ")
    for split in split_string_list:
        if split =="":
            pass
        else:
            if ("(" in split) or (")" in split): ## The data contain brackets that is not useful
                pass
            else:
                dummy_append.append(float(split))
    dummy_df_row = [dummy_append[1],dummy_append[4],dummy_append[2]]
    obs_onebody_df.loc[counter] = dummy_df_row

obs_onebody_df.to_csv(filenames['output_1B_source_csv'])

In [11]:
## config particles and orbitals number
with open(filenames['config_output_py'],'w') as myfile:
    myfile.write("num_particles = "+ str(num_particles))
    myfile.write("\nnum_nucleon_orbitals = "+ str(num_nucleon_orbitals))
    myfile.write("\nnum_spin_orbitals = "+ str(num_spin_orbitals))

In [ ]:
## to improve next time: 
## connect the gap between this to producing proper matrix elements that is fed directly into the VQE algorithm.

In [ ]:
obs_twobody_df
obs_onebody_df